In [156]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


In [157]:
root = 'data/'
results_merged = pd.read_csv(root + 'results_merged.csv')

/var/folders/gy/c72q5khn4jv42tkd5s9tmlkr0000gn/T/ipykernel_9678/888726719.py:2: DtypeWarning: Columns (4,31) have mixed types. Specify dtype option on import or set low_memory=False.
  results_merged = pd.read_csv(root + 'results_merged.csv')


In [158]:
results_merged.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time_x,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,status,year,round,circuitId,name_x,date,time_y,circuitRef,name_y,location,country,lat,lng,alt,url_y
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,Finished,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,Finished,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,Finished,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,Finished,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,Finished,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...


In [159]:
results_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26080 entries, 0 to 26079
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         26080 non-null  int64  
 1   raceId           26080 non-null  int64  
 2   driverId         26080 non-null  int64  
 3   constructorId    26080 non-null  int64  
 4   number           26080 non-null  object 
 5   grid             26080 non-null  int64  
 6   position         26080 non-null  object 
 7   positionText     26080 non-null  object 
 8   positionOrder    26080 non-null  int64  
 9   points           26080 non-null  float64
 10  laps             26080 non-null  int64  
 11  time_x           26080 non-null  object 
 12  milliseconds     26080 non-null  object 
 13  fastestLap       26080 non-null  object 
 14  rank             26080 non-null  object 
 15  fastestLapTime   26080 non-null  object 
 16  fastestLapSpeed  26080 non-null  object 
 17  statusId    

In [160]:
results_merged.shape

(26080, 33)

In [161]:
results_merged.drop(['number', 'positionText', 'positionOrder', 'points', 'laps', 'time_x',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'status', 'round', 'name_x', 'date', 'time_y', 'circuitRef', 'name_y', 'location',
       'country',  'url_y', 'lat', 'lng', 'resultId' ], axis=1, inplace=True)

In [162]:
pd.set_option('display.max_columns', None)
results_merged.head()

,raceId,driverId,constructorId,grid,position,statusId,year,circuitId,alt
0,18,1,1,1,1,1,2008,1,10
1,18,2,2,5,2,1,2008,1,10
2,18,3,3,7,3,1,2008,1,10
3,18,4,4,11,4,1,2008,1,10
4,18,5,1,3,5,1,2008,1,10


In [163]:
# Change "\N" in position to 100
results_merged['position'] = results_merged['position'].replace('\\N', '35')

In [164]:
results_merged['position'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '35', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '23',
       '27', '30', '24', '26', '25', '32', '28', '29', '31', '22', '33'],
      dtype=object)

In [165]:
results_merged.replace('\\N', pd.NA, inplace=True)

In [166]:
results_merged.isna().sum()

raceId            0
driverId          0
constructorId     0
grid              0
position          0
statusId          0
year              0
circuitId         0
alt              60
dtype: int64

In [167]:
# Drop rows with missing values
results_merged.dropna(inplace=True)

In [168]:
results_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26020 entries, 0 to 26079
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   raceId         26020 non-null  int64 
 1   driverId       26020 non-null  int64 
 2   constructorId  26020 non-null  int64 
 3   grid           26020 non-null  int64 
 4   position       26020 non-null  object
 5   statusId       26020 non-null  int64 
 6   year           26020 non-null  int64 
 7   circuitId      26020 non-null  int64 
 8   alt            26020 non-null  object
dtypes: int64(7), object(2)
memory usage: 2.0+ MB


In [169]:
# results_merged = pd.get_dummies(results_merged, columns=['raceId', 'driverId', 'constructorId', 'grid', 'statusId', 'raceId', 'circuitId'])

In [170]:
# Define features and target variable

features = ['raceId', 'driverId', 'constructorId', 'grid', 'statusId', 'year', 'circuitId', 'alt']
target = 'position'

In [171]:
# Split the features into numerical and categorical
numeric_features = ['grid', 'statusId', 'year', 'alt']
categorical_features = ['raceId', 'driverId', 'constructorId', 'circuitId'] 

In [172]:
# Define the preprocessing pipelines for both numerical and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [173]:
# Split the dataset into train and test sets
X = results_merged[features]
y = results_merged[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [174]:
# Fit the preprocessing pipeline and transform the data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [175]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_processed.astype('float32').toarray())
y_train_tensor = torch.tensor(y_train.astype('float32').values.reshape(-1, 1))
X_test_tensor = torch.tensor(X_test_processed.astype('float32').toarray())
y_test_tensor = torch.tensor(y_test.astype('float32').values.reshape(-1, 1))

In [204]:
# Define the model architecture
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [205]:
# Initialize the model
model = RegressionModel(X_train_tensor.shape[1])

In [206]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [207]:
# Train the model
num_epochs = 100
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        targets = y_train_tensor[i:i+batch_size]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = torch.sqrt(criterion(outputs, targets))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], RMSE Loss: {loss.item():.4f}')

Epoch [1/100], RMSE Loss: 12.3973
Epoch [2/100], RMSE Loss: 11.6925
Epoch [3/100], RMSE Loss: 10.9693
Epoch [4/100], RMSE Loss: 10.3597
Epoch [5/100], RMSE Loss: 9.7793
Epoch [6/100], RMSE Loss: 9.1670
Epoch [7/100], RMSE Loss: 8.7041
Epoch [8/100], RMSE Loss: 8.2169
Epoch [9/100], RMSE Loss: 7.7339
Epoch [10/100], RMSE Loss: 7.2951
Epoch [11/100], RMSE Loss: 6.9770
Epoch [12/100], RMSE Loss: 6.7696
Epoch [13/100], RMSE Loss: 6.3392
Epoch [14/100], RMSE Loss: 5.4742
Epoch [15/100], RMSE Loss: 4.5050
Epoch [16/100], RMSE Loss: 3.9597
Epoch [17/100], RMSE Loss: 3.0948
Epoch [18/100], RMSE Loss: 2.3140
Epoch [19/100], RMSE Loss: 1.9025
Epoch [20/100], RMSE Loss: 1.4816
Epoch [21/100], RMSE Loss: 1.1105
Epoch [22/100], RMSE Loss: 1.1570
Epoch [23/100], RMSE Loss: 1.1816
Epoch [24/100], RMSE Loss: 1.0339
Epoch [25/100], RMSE Loss: 1.1978
Epoch [26/100], RMSE Loss: 1.0147
Epoch [27/100], RMSE Loss: 1.3295
Epoch [28/100], RMSE Loss: 0.8882
Epoch [29/100], RMSE Loss: 1.7368
Epoch [30/100], RMS

In [208]:
# Evaluate the model
with torch.no_grad():
    y_pred = model(X_test_tensor)
    rmse = np.sqrt(criterion(y_pred, y_test_tensor))
    print(f"Root Mean Squared Error: {rmse:.4f}")

Root Mean Squared Error: 8.0641


In [211]:
# print the prediction for the year 2023
X_2023 = X_test_tensor

ValueError: Specifying the columns using strings is only supported for pandas DataFrames